# Qumode VQE for an eight-qubit Hamiltonian

Explore `TensorFlow` optimizers for VQE.

## Prerequisite

Installation cells for Google Colab users.

In [ ]:
!pip install qutip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 54.0 MB/s eta 0:00:00


In [ ]:
!pip install tensorflow_probability==0.23.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 15.0 MB/s eta 0:00:00
  Attempting uninstall: tensorflow_probability
    Found existing installation: tensorflow-probability 0.24.0
    Uninstalling tensorflow-probability-0.24.0:
      Successfully uninstalled tensorflow-probability-0.24.0


In [ ]:
!pip install silence-tensorflow

  Preparing metadata (setup.py) ... done
  Created wheel for silence-tensorflow: filename=silence_tensorflow-1.2.3-py3-none-any.whl size=6749 sha256=c2c54c7027c85954e2c8f7ad74995f6852ed3f273130a00071f6c41127809250
  Stored in directory: /root/.cache/pip/wheels/2e/91/a1/2d32c0ea21439c6367fe1acaa2d3a0377a95ae51cf47c13521
Successfully built silence-tensorflow


In [ ]:
!pip install openfermion
!pip install openfermionpyscf

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 56.4 MB/s eta 0:00:00
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13819 sha256=b45be329d923e03d7e7ba75218a0ac2ff284f05d0098bc1fbea2d16254e28406
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 25.7 MB/s eta 0:00:00


Import libaries.

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

In [ ]:
import qutip as qt
import tensorflow as tf
import tensorflow_probability as tfp

In [ ]:
from openfermion.chem import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner, bravyi_kitaev
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.ops.representations import get_tensors_from_integrals
from openfermionpyscf import run_pyscf

In [ ]:
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [ ]:
!git clone https://github.com/rishabdchem/qumode_est_paper/

import sys
sys.path.append('./qumode_est_paper/')
%cd qumode_est_paper/

Mounted at /content/drive


## Circuits

### Basics

In [ ]:
def qt2tf(qt_object, dtype=tf.complex128):
    if tf.is_tensor(qt_object) or qt_object is None:
        return qt_object
    return tf.constant(qt_object.full(), dtype=dtype)


def get_cvec_tf(r, theta, dtype=tf.complex128):
    real_part = r * tf.cos(theta)
    imag_part = r * tf.sin(theta)
    return tf.complex(real_part, imag_part)


def get_cvec_np(r, theta):
    r = np.array(r)
    theta = np.array(theta)
    return r * np.exp(1j * theta)

In [ ]:
def qproj00_qt():
    return qt.basis(2, 0).proj()


def qproj11_qt():
    return qt.basis(2, 1).proj()


def qproj01_qt():
    op = np.array([[0, 1], [0, 0]])
    return qt.Qobj(op)


def qproj10_qt():
    op = np.array([[0, 0], [1, 0]])
    return qt.Qobj(op)


def hadamard_qt():
    op = (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]])
    return qt.Qobj(op)

### SNAP-displacement

We will use `QuTip` to generate SNAP-displacement ansatz.

In [ ]:
def unpack_params_snap_disp(X, nfock):
    # Initialize
    ndepth = X.shape[0] // (nfock + 1)

    # Unpack
    alpha = X[:ndepth].copy()
    d1 = ndepth * nfock
    theta = X[ndepth:ndepth+d1].reshape((ndepth, nfock))

    return alpha, theta


def pack_params_snap_disp(alpha, theta):
    # Initialize
    ndepth = alpha.shape[0]
    nfock = theta.shape[1]
    dim = (nfock + 1) * ndepth
    X = np.zeros((dim,))

    # Pack
    X[:ndepth] = alpha.copy()
    d1 = ndepth * nfock
    X[ndepth:ndepth+d1] = theta.reshape(-1)

    return X

Selective number-dependent arbitray phase (SNAP) and displacement operator ([reference](https://doi.org/10.1103/PhysRevA.92.040303))

\begin{align*}
U (\alpha, \vec{\theta})
&= S (\vec{\theta}) \: D (\alpha),
\\
S (\vec{\theta})
&= \sum_{n = 0}^{L - 1} \: \exp ( i \: \theta_n ) \: |n \rangle \langle n|,
\\
D (\alpha)
&= e^{ \alpha \: ( a^\dagger - a ) }.
\end{align*}

In [ ]:
def snap_disp_op_qt(alpha, thetavec):
    """
    SNAP-displacement operator.

    Arguments:
    alpha -- displacement coefficient
    thetavec -- SNAP parameters
    """
    # Initialize
    nfock = thetavec.shape[0]

    # SNAP
    S2 = np.exp(1j * thetavec[0]) * qt.basis(nfock, 0).proj()
    for i in range(1, nfock):
        S2 += np.exp(1j * thetavec[i]) * qt.basis(nfock, i).proj()

    # Displacement
    D2 = qt.displace(nfock, alpha)

    return S2 * D2

Build the ansatz matrix of depth $N_d$

$$ \mathcal{U} (\vec{\alpha}, \bar{\theta})
= U (\alpha_{N_d}, \vec{\theta}_{N_d}) \cdots
U (\alpha_1, \vec{\theta}_1),
$$

where $\vec{\alpha}$ is an $N_d$-dimensional vector and
$ \bar{\theta}_{N_d \times L} $ is a matrix.


In [ ]:
def snap_disp_ansatz_qt(Xvec, nfock):
    """
    SNAP-displacement ansatz.

    Arguments:
    Xvec -- ansatz parameters
    nfock -- Fock cutoff
    """
    # Initialize
    alphavec, thetamat = unpack_params_snap_disp(Xvec, nfock)
    ndepth = thetamat.shape[0]
    uni = snap_disp_op_qt(alphavec[0], thetamat[0, :])

    # Check
    if ndepth == 1:
        return uni.full()

    # Loop through blocks
    for i in range(1, ndepth):
        new_uni = snap_disp_op_qt(alphavec[i], thetamat[i, :])
        uni = ( new_uni * uni )

    return uni.full()

### TensorFlow

In [ ]:
def identity_tf(N, dtype=tf.complex128):
    return tf.eye(N, dtype=dtype)


def destroy_tf(N, dtype=tf.complex128):
    a = tf.linalg.diag(tf.sqrt(tf.range(1, N, dtype=tf.float64)), k=1)
    return tf.cast(a, dtype=dtype)


def create_tf(N, dtype=tf.complex128):
    return tf.cast(tf.linalg.adjoint(destroy_tf(N, dtype)), dtype=dtype)


def num_proj_tf(N, j, dtype=tf.complex128):
    op = qt.basis(N, j).proj()
    return qt2tf(op, dtype)

In [ ]:
def displace_tf(N, alpha, dtype=tf.complex128):
    """
    Complex-valued matrix representation of

    D (alpha) = exp( alpha b! - alpha* b ).

    Arguments:
    alpha -- displacement parameter
    N -- qumode dimension
    dtype -- data type
    """
    gen = ( tf.cast(alpha, dtype=dtype) * create_tf(N, dtype) )
    gen -= ( tf.cast(tf.math.conj(alpha), dtype=dtype) * destroy_tf(N, dtype) )

    return tf.cast(tf.linalg.expm(gen), dtype=dtype)

In [ ]:
def snap_disp_op_tf(alpha, thetavec, nfock, dtype=tf.complex128):
    """
    SNAP-displacement operator.

    Arguments:
    alpha -- displacement coefficient
    thetavec -- SNAP parameters
    """
    # SNAP term
    S2 = tf.exp( 1j * tf.cast( thetavec[0], dtype=dtype) ) * num_proj_tf(nfock, 0, dtype=dtype)
    for i in range(1, nfock):
        S2 += tf.exp( 1j * tf.cast( thetavec[i], dtype=dtype) ) * num_proj_tf(nfock, i, dtype=dtype)

    # Displacement term
    D2 = displace_tf(nfock, alpha, dtype=dtype)

    # Return the combined operator
    return tf.cast(tf.linalg.matmul(S2, D2), dtype=dtype)

In [ ]:
def beam_splitter_tf(theta, phi, nfocks, dtype=tf.complex128):
    """
    BS (theta, phi) = exp[ i (theta/2) ( exp(i phi) b1! b2 + h.c. ) ].

    Arguments:
    theta, phi -- rotation parameters
    nfocks -- Fock cutoffs
    """
    # Operators
    b1 = tf.linalg.LinearOperatorFullMatrix(destroy_tf(nfocks[0], dtype=dtype), dtype)
    b2 = tf.linalg.LinearOperatorFullMatrix(destroy_tf(nfocks[1], dtype=dtype), dtype)
    b1dag = tf.linalg.LinearOperatorFullMatrix(create_tf(nfocks[0], dtype=dtype), dtype)
    b2dag = tf.linalg.LinearOperatorFullMatrix(create_tf(nfocks[1], dtype=dtype), dtype)

    # Tensor products
    op1 = tf.linalg.LinearOperatorKronecker([b1dag, b2]).to_dense()
    op2 = tf.linalg.LinearOperatorKronecker([b1, b2dag]).to_dense()

    # Generator of the beam splitter operator
    gen = tf.exp(1j * tf.cast(phi, dtype=dtype)) * op1
    gen += tf.exp(-1j * tf.cast(phi, dtype=dtype)) * op2

    # Hamiltonian
    H = 1j * (tf.cast(theta, dtype=dtype) / 2) * gen

    # Unitary
    return tf.cast(tf.linalg.expm(H), dtype=dtype)

### Multimode SNAP-displacement

SNAP-displacement gates with beamsplitter for two qumodes

\begin{align*}
U (\beta, \phi, \vec{\alpha}, \vec{\theta}_1, \vec{\theta}_2)
&= \big[ S (\vec{\theta}_1) \: D (\alpha_1) \otimes I \big] \:
\big[ I \otimes S (\vec{\theta}_2) \: D (\alpha_2) \big] \:
BS_{1, 2} (\beta, \phi),
\\
BS_{1, 2} (\beta, \phi)
&= e^{ i \frac{\beta}{2} \big(
e^{i \phi} a_1^\dagger a_2 + \text{h.c.} \big)},
\end{align*}
where $\vec{\alpha}$ is a two-dimensional vector,
$\vec{\theta}_1$ is an $L_1$-dimensional vector, $\vec{\theta}_2$ is an $L_2$-dimensional vector.


In [ ]:
def multimode_snap_disp_op_tf(beta, phi, alpha_vec, theta_vec1, theta_vec2, nfocks, dtype=tf.complex128):
    """
    Qumode-qumode SNAP-displacement.

    Arguments:
    beta, phi -- beamsplitter parameters
    alpha_vec -- displacement parameters
    theta_vec1, theta_vec2 -- SNAP parameters
    """
    # SNAP-displacement operators
    op1 = snap_disp_op_tf(alpha_vec[0], theta_vec1, nfocks[0], dtype=dtype)
    op2 = snap_disp_op_tf(alpha_vec[1], theta_vec2, nfocks[1], dtype=dtype)

    # Matrices
    op1_mat = tf.linalg.LinearOperatorFullMatrix(op1, dtype)
    op2_mat = tf.linalg.LinearOperatorFullMatrix(op2, dtype)

    # Identity operators
    identity1 = tf.linalg.LinearOperatorIdentity(nfocks[0], dtype=dtype)
    identity2 = tf.linalg.LinearOperatorIdentity(nfocks[1], dtype=dtype)

    # Kronecker products
    SD1 = tf.linalg.LinearOperatorKronecker([op1_mat, identity2]).to_dense()
    SD2 = tf.linalg.LinearOperatorKronecker([identity1, op2_mat]).to_dense()

    # Kronecker product result
    SDtwo = tf.linalg.matmul(SD2, SD1)

    # Beamsplitter
    BS = beam_splitter_tf(beta, phi, nfocks, dtype=dtype)

    return tf.cast(tf.linalg.matmul(BS, SDtwo), dtype=dtype)

Build the ansatz matrix of depth $N_d$

$$ \mathcal{U} (\vec{\beta}, \vec{\phi}, \bar{\alpha},
\bar{\theta}^{(1)}, \bar{\theta}^{(2)})
= U (\beta_{N_d}, \phi_{N_d}, \vec{\alpha}_{N_d}, \vec{\theta}_{N_d}^{(1)}, \vec{\theta}_{N_d}^{(2)})
\: \cdots \:
U (\beta_1, \phi_1, \vec{\alpha}_1, \vec{\theta}_1^{(1)}, \vec{\theta}_1^{(2)}),
$$
where $\vec{\beta}, \vec{\phi}$ are $N_d$-dimensional vectors, $\bar{\alpha}$ is a matrix of dimensions $N_d \times 2$,
$\bar{\theta}^{(1)}$ is a matrix of dimensions $N_d \times L_1$, and
$\bar{\theta}^{(2)}$ is a matrix of dimensions $N_d \times L_2$.

In [ ]:
def pack_params_ansatz(beta, phi, alpha_vec, theta_vec1, theta_vec2):
    # Flatten each parameter tensor into a 1D tensor (vector)
    beta_flat = tf.reshape(beta, [-1])
    phi_flat = tf.reshape(phi, [-1])
    alpha_flat = tf.reshape(alpha_vec, [-1])  # Flatten the alpha matrix
    theta1_flat = tf.reshape(theta_vec1, [-1])  # Flatten the theta1 matrix
    theta2_flat = tf.reshape(theta_vec2, [-1])  # Flatten the theta2 matrix

    # Concatenate all flattened tensors into a single vector
    packed_vec = tf.concat([beta_flat, phi_flat, alpha_flat, theta1_flat, theta2_flat], axis=0)

    return tf.Variable(packed_vec)


def unpack_params_ansatz(packed_vec, ndepth, L1, L2):
    # Extract the lengths of each individual parameter
    beta_size = ndepth
    phi_size = ndepth
    alpha_size = ndepth * 2
    theta1_size = ndepth * L1
    theta2_size = ndepth * L2

    # Unpack the vector into its original components
    beta = tf.reshape(packed_vec[:beta_size], [beta_size])
    phi = tf.reshape(packed_vec[beta_size:beta_size + phi_size], [phi_size])
    alpha_vec = tf.reshape(packed_vec[beta_size + phi_size:beta_size + phi_size + alpha_size], [ndepth, 2])
    theta_vec1 = tf.reshape(packed_vec[beta_size + phi_size + alpha_size:beta_size + phi_size + alpha_size + theta1_size], [ndepth, L1])
    theta_vec2 = tf.reshape(packed_vec[beta_size + phi_size + alpha_size + theta1_size:], [ndepth, L2])

    return beta, phi, alpha_vec, theta_vec1, theta_vec2

In [ ]:
def multimode_snap_disp_tf(Xvec, ndepth, nfocks, dtype=tf.complex128):
    # Initialize
    beta, phi, alpha_vec, theta_vec1, theta_vec2 = unpack_params_ansatz(Xvec, ndepth, nfocks[0], nfocks[1])
    uni = multimode_snap_disp_op_tf(beta[0], phi[0], alpha_vec[0, :], theta_vec1[0, :],
                                    theta_vec2[0, :], nfocks, dtype=dtype)

    # Check
    if ndepth == 1:
        return tf.cast(uni, dtype=dtype)

    # Loop through blocks
    for i in range(1, ndepth):
        new_mat = multimode_snap_disp_op_tf(beta[i], phi[i], alpha_vec[i, :],
                                            theta_vec1[i, :], theta_vec2[i, :],
                                            nfocks, dtype=dtype)
        uni = tf.linalg.matmul(new_mat, uni)

    return tf.cast(uni, dtype=dtype)

## Qubit operators

### General

`OpenFermion` functions.

In [ ]:
def append_ids(qubitstr, nqubits):
    """
    Modify a qubit string so that identities are explicity included.

    Arguments:
    qubitstr -- Input qubit string as a tuple of tuples
    nqubits  -- Number of qubits
    """
    # Initialize a tuple
    newstr = [None] * nqubits

    # Copy
    for p in range(len(qubitstr)):
        newstr[qubitstr[p][0]] = qubitstr[p]

    # Updates
    for p in range(nqubits):
        if newstr[p] == None:
            newstr[p] = (p, 'I')

    return newstr

In [ ]:
def qubit_partition(qubitop, nqubit, quditvec):
    """
    Transform one qubit string to a set of a qubit strings based on
    the partition chosen.

    Arguments:
    qubitop  -- A single Pauli operator string as a tuple of tuples
    nqubit   -- Number of qubit operators including identity
    quditvec -- Qudit vector for reorganizing qubit Hilbert space
    """
    # Check
    if sum(quditvec) != nqubit:
        raise ValueError("Wrong qudit vector elements")

    # A list of tuples
    qubitstr = append_ids(qubitop, nqubit)

    # New qubit strings
    newstrings = []
    for j in range(len(quditvec)):
        # Initialize
        templist = []
        # Loop for string as a list
        for p in range(quditvec[j]):
            # Tuple
            temptuple = (p, qubitstr[sum(quditvec[:j]) + p][1])
            # Add to list
            templist.append(temptuple)
        # Update list of strings
        newstrings.append(templist)

    return newstrings

In [ ]:
def simple_qubit_string(qubitop, nqubit, quditvec):
    """
    Transform one qubit string to a set of a qubit strings based on
    the partition chosen.

    Arguments:
    qubitop  -- A single Pauli operator string as a tuple of tuples
    nqubit   -- Number of qubit operators including identity
    quditvec -- Qudit vector for reorganizing qubit Hilbert space
    """
    # Check
    if sum(quditvec) != nqubit:
        raise ValueError("Wrong qudit vector elements")

    # A list of tuples
    qubitstr = qubit_partition(qubitop, nqubit, quditvec)

    # New qubit strings
    string_list = []
    for j in range(len(quditvec)):
        string_list.append(''.join(op for _, op in qubitstr[j]))

    return string_list

In [ ]:
def qubit_string_from_openfermion(qubitop, nqubit, quditvec):
    """
    Transform a QubitOperator to a set of a qubit strings and coefficients based on
    the partition chosen.

    Arguments:
    qubitop  -- QubitOperator
    nqubit   -- Number of qubit operators including identity
    quditvec -- Qudit vector for reorganizing qubit Hilbert space
    """
    # Check
    if sum(quditvec) != nqubit:
        raise ValueError("Wrong qudit vector elements")

    # Get data
    qubit_strings = list(qubitop.terms.keys())
    coeffs = np.array(list( qubitop.terms.values() ))

    # New qubit strings
    ops_list = []
    for qubit_op in qubit_strings:
        ops_list.append( simple_qubit_string(qubit_op, nqubit, quditvec) )

    return ops_list, coeffs

`QuTip` functions.

In [ ]:
def generate_tensor_product(string):
    """
    Generate QuTip object given a string representing a Pauli word.
    """
    # Define a mapping of characters to corresponding QuTiP operators
    operator_map = {
        'I': qt.qeye(2),  # Identity operator
        'X': qt.sigmax(),  # Pauli-X operator
        'Y': qt.sigmay(),  # Pauli-Y operator
        'Z': qt.sigmaz()   # Pauli-Z operator
    }

    # Create a list to collect the operators
    operators = []

    # Append the corresponding operators based on the input string
    for char in string:
        operators.append(operator_map[char])

    # Compute the tensor product of all operators in the list
    U = qt.tensor(*operators).full()

    return qt.Qobj(U)

### Specific

In [ ]:
def snap_uni_four_pauli(pword, data_dict):
    """
    Get SNAP-displacement unitary with Fock cutoff = 16.

    Arguments:
    pword -- four qubit Pauli word
    data_dict -- parameter dictionary
    """
    nfock = 16
    Xvec = np.array( data_dict[pword][2] )
    U = snap_disp_ansatz_qt(Xvec, nfock)

    return U

In [ ]:
def snap_eight_qubit_ham(qubit_op, data_dict):
    """
    Get a list of SNAP-displacement unitary pairs (U1, U2),
    each with Fock cutoff = 16.

    Arguments:
    qubit_op -- eight-qubit openfermion QubitOperator
    data_dict -- parameter dictionary
    """
    # Initialize
    nqubit = 8
    nfock = 16
    nqpart = 4
    quditvec = [nqpart, nqpart,]

    # String and coefficient lists
    ops_list, coeffs = qubit_string_from_openfermion(qubit_op, nqubit, quditvec)

    # Final
    U1_lists = []
    U2_lists = []
    for i in range(len(coeffs)):
        if ops_list[i][0] == 'I' * nqpart:
            U1_lists.append( qt.qeye(nfock) )
        else:
            U1_lists.append( snap_uni_four_pauli(ops_list[i][0], data_dict) )
        if ops_list[i][1] == 'I' * nqpart:
            U2_lists.append( qt.qeye(nfock) )
        else:
            U2_lists.append( snap_uni_four_pauli(ops_list[i][1], data_dict) )

    return U1_lists, U2_lists, coeffs

In [ ]:
def pauli_eight_qubit_ham(qubit_op):
    """
    Build the eight-qubit Hamiltonian.

    Argument:
    qubit_op -- eight-qubit openfermion QubitOperator
    """
    # Initialize
    nqubit = 8
    nfock = 16
    quditvec = [nqubit,]

    # String and coefficient lists
    ops_list, coeffs = qubit_string_from_openfermion(qubit_op, nqubit, quditvec)

    # Final
    ham = []
    for i in range(len(coeffs)):
        term = (coeffs[i] * generate_tensor_product(ops_list[i][0]) )
        ham.append(term)

    return sum(ham)

### Hamiltonians

In [ ]:
def ham_linear_sym_h4(R, f2q='JWT'):
    """
    (0) -- (R) -- (2R) -- (3R)

    Arguments:
    R -- bond distance in Angstrom
    f2q -- Fermion to qubit mapping
    """
    # Define parameters
    basis = "sto-3g"
    multiplicity = 1
    charge = 0

    # XYZ coordinates
    geometry = [("H", (0, 0, 0)), \
                ("H", (0, 0, R)), \
                ("H", (0, 0, 2*R)), \
                ("H", (0, 0, 3*R))]

    # Define molecule
    molecule = MolecularData(geometry, basis, multiplicity, charge)

    # Hartree-Fock
    hf_molecule = run_pyscf(molecule, run_scf=1)

    # Fermionic Hamiltonian
    ham_fermi = get_fermion_operator(hf_molecule.get_molecular_hamiltonian())

    # Qubit mapping
    if f2q == 'JWT':
        ham = jordan_wigner(ham_fermi)
    elif f2q == 'BKT':
        ham = bravyi_kitaev(ham_fermi)

    return ham

In [ ]:
H = ham_linear_sym_h4(0.7)

In [ ]:
len(H.terms)

185

In [ ]:
def fci_linear_sym_h4(R):
    """
    (0) -- (R) -- (2R) -- (3R)

    Argument:
    R -- bond distance in Angstrom
    """
    # Define parameters
    basis = "sto-3g"
    multiplicity = 1
    charge = 0

    # XYZ coordinates
    geometry = [("H", (0, 0, 0)), \
                ("H", (0, 0, R)), \
                ("H", (0, 0, 2*R)), \
                ("H", (0, 0, 3*R))]

    # Define molecule
    molecule = MolecularData(geometry, basis, multiplicity, charge)

    # Run HF calculation
    molecule = run_pyscf(molecule, run_fci=1)

    return molecule.hf_energy, molecule.fci_energy

## Expectation value

### Hadamard test

In [ ]:
def trace_out_qumodes_tf(state, nfocks):
    """
    Performs a partial trace over the qumodes of a density matrix,
    leaving the reduced density matrix for the qubit.

    Arguments:
    state -- statevector for qubit-qumode-qumode system in TensorFlow
    nfock -- Fock cutoffs for two qumodes
    """
    # Initialize
    rho = tf.matmul( state, tf.transpose(tf.math.conj(state)) )

    # Dimension matching
    rho_reshaped = tf.reshape(rho, [2, nfocks[0], nfocks[1], 2, nfocks[0], nfocks[1]])

    # Qubit RDM
    rho_qubit = tf.einsum('ijkljk->il', rho_reshaped)

    return rho_qubit

In [ ]:
def include_hadamard(U1, U2):
    # Fock cutoffs
    nfocks = []
    nfocks.append(U1.shape[0])
    nfocks.append(U2.shape[0])

    # U1 part
    op1 = qt.tensor(hadamard_qt(), qt.qeye(nfocks[0]), qt.qeye(nfocks[1]))
    op2 = qt.tensor(qt.basis(2, 0).proj(), qt.qeye(nfocks[0]), qt.qeye(nfocks[1]))
    op2 += qt.tensor(qt.basis(2, 1).proj(), U1, qt.qeye(nfocks[1]))
    U1_op = op1 * op2 * op1

    # U2 part
    op3 = qt.tensor(hadamard_qt(), qt.qeye(nfocks[0]), qt.qeye(nfocks[1]))
    op4 = qt.tensor(qt.basis(2, 0).proj(), qt.qeye(nfocks[0]), qt.qeye(nfocks[1]))
    op4 += qt.tensor(qt.basis(2, 1).proj(), qt.qeye(nfocks[0]), U2)
    U2_op = op3 * op4 * op3

    # Final
    U = (U1_op * U2_op).full()

    return qt.Qobj(U)

In [ ]:
def hadamard_ops_tf(U1_list, U2_list, dtype=tf.complex128):
    """
    T(j) = ( U1 (j) x I ) ( I x U2 (j) )

    Arguments:
    U1_list, U2_list -- list of four-qubit unitaries
    """
    # Initialize
    ns = len(U1_list)

    # Loop
    T_list = []
    for i in range(ns):
        U1_op = qt.Qobj(U1_list[i])
        U2_op = qt.Qobj(U2_list[i])
        T_list.append( qt2tf( include_hadamard(U1_op, U2_op), dtype=dtype) )

    return T_list

In [ ]:
@tf.function
def psi_uni_psi_hadamard_tf(state, T2, Zop, nfocks, dtype=tf.complex128):
    """
    Expectation value Re( <state | ( U1 x I ) ( I x U2 ) | state> ).

    Arguments:
    state -- qubit-qumode-qumode state in TensorFlow
    T2 -- Hadamard-included operator
    dtype -- data type
    """
    # T |state>
    psi = tf.linalg.matmul(T2, state)

    # Extract qubit RDM
    rho_qubit = trace_out_qumodes_tf(psi, nfocks)

    # <Z>
    ov = tf.linalg.trace(tf.matmul(rho_qubit, Zop))

    return tf.math.real(ov)

### Loss function

In [ ]:
def state_from_multimode_snap_tf(Xvec, ndepth, nfocks, vac, dtype=tf.complex128):
    """
    Aux-qubit two-qumode state |Psi> = |0> x ( U |0, 0> ).

    Arguments:
    Xvec -- ansatz parameters
    ndepth -- circuit depth
    nfocks -- Fock cutoffs for two qumodes
    vac -- vacuum state for qubit-qumode-qumode
    dtype -- data type
    """
    # Ansatz unitary
    U = multimode_snap_disp_tf(Xvec, ndepth, nfocks, dtype=dtype)

    # I x U
    id_op = tf.linalg.LinearOperatorIdentity(2, dtype=dtype)
    U_op = tf.linalg.LinearOperatorFullMatrix(U, dtype)
    op = tf.linalg.LinearOperatorKronecker([id_op, U_op]).to_dense()

    # ( I x U ) (|0> x |0, 0> )
    psi = tf.cast(tf.linalg.matmul(op, vac), dtype=dtype)

    return psi

In [ ]:
@tf.function
def energy_val_tf(psi, Zop, nfocks, T_list, coeffs):
    """
    Compute sum(j) C(j) Re( <psi | ( U1 (j) x I ) ( I x U2 (j) ) | psi> ).

    Arguments:
    psi -- trial state
    ndepth -- circuit depth
    T_list -- list of four-qubit QuTip unitaries
    coeffs - coefficients
    nfocks -- Fock cutoffs
    """
    # Convert T_list to a Tensor if it's a list
    T_list_tf = tf.convert_to_tensor(T_list, dtype=tf.complex128)

    # Assuming `compute_term` is defined properly to use T_list_tf
    def compute_term(i):
        coeff_tf = tf.cast( tf.gather(coeffs, i), tf.float64 )
        term_value = coeff_tf * psi_uni_psi_hadamard_tf(psi, T_list_tf[i], Zop, nfocks)  # Use T_list_tf here
        return term_value

    # Use tf.map_fn to compute terms in parallel
    terms = tf.map_fn(compute_term, tf.range(tf.shape(coeffs)[0]), fn_output_signature=tf.float64)

    # Compute the energy as the sum of the terms
    en = tf.reduce_sum(terms)

    return en

## Optimize

In [ ]:
def snap_vqe_tfp(T_list, coeffs, ndepth, niter=100, threshold=1e-08, Xvec=None):
    # Initialize
    nfocks = [16, 16]
    vac_qt = qt.tensor( qt.basis(2, 0), qt.basis(nfocks[0], 0), qt.basis(nfocks[1], 0) )
    vac = qt2tf(vac_qt)
    P0 = qt2tf( qt.tensor( qt.basis(2, 0).proj(), qt.qeye(nfocks[0]), qt.qeye(nfocks[1]) ) )
    Zop = qt2tf(qt.sigmaz())

    # Parameter shapes
    shape1 = (ndepth,)
    shape2 = (ndepth, 2,)
    shape3 = (ndepth, nfocks[0],)
    shape4 = (ndepth, nfocks[1],)

    # Guess
    if Xvec is None:
        beta = tf.Variable(tf.random.uniform(shape1, minval=0, maxval=np.pi, dtype=tf.float64))
        phi = tf.Variable(tf.random.uniform(shape1, minval=0, maxval=np.pi, dtype=tf.float64))
        alpha = tf.Variable(tf.random.uniform(shape2, minval=-3.0, maxval=3.0, dtype=tf.float64))
        theta1 = tf.Variable(tf.random.uniform(shape3, minval=0, maxval=np.pi, dtype=tf.float64))
        theta2 = tf.Variable(tf.random.uniform(shape3, minval=0, maxval=np.pi, dtype=tf.float64))
        Xvec = pack_params_ansatz(beta, phi, alpha, theta1, theta2)
    else:
        Xvec = tf.Variable(Xvec, dtype=tf.float64)  # Ensure Xvec is a Variable

    # Loss function
    @tf.function
    def loss_fun(Xvec):
        psi = state_from_multimode_snap_tf(Xvec, ndepth, nfocks, vac, dtype=tf.complex128)
        return energy_val_tf(psi, Zop, nfocks, T_list, coeffs)

    # Gradient function
    @tf.function
    def value_and_gradients_function(Xvec):
        with tf.GradientTape() as tape:
            tape.watch(Xvec)
            loss = loss_fun(Xvec)
        gradients = tape.gradient(loss, Xvec)
        return loss, gradients

    # Run BFGS optimization
    result = tfp.optimizer.bfgs_minimize(
        value_and_gradients_function,
        initial_position=Xvec,
        tolerance=threshold,
        max_iterations=niter
    )

    # Final result
    final_loss = loss_fun(result.position)

    return final_loss, result.position, result

## Explore

Load the dictionary.

In [ ]:
fname = 'snap-four-qubit-dict_nd16.npy'

op_dict = np.load(fname, allow_pickle=True).item()

Get `QubitOperator` for the Hamiltonian of linear equidistant H$_4$ molcule in minimal basis.

In [ ]:
hhdis = 0.5 # in Angstrom

In [ ]:
fci_linear_sym_h4(hhdis)

(-1.628609703029901, -1.6531169519401132)

In [ ]:
ham_op = ham_linear_sym_h4(hhdis)

Get SNAP-displacement unitaries.

In [ ]:
U1_list, U2_list, coeffs = snap_eight_qubit_ham(ham_op, op_dict)

cvec = tf.constant( np.real(coeffs), dtype=tf.float64 )
T_list = hadamard_ops_tf(U1_list, U2_list)

VQE.

In [ ]:
ndepth = 20

start_time = time.time()

en, Xvec, result = snap_vqe_tfp(T_list, cvec, ndepth, niter=2000,
                                threshold=1e-12)

end_time = time.time()

print("en: ", en.numpy())
print("iters: ", result.num_iterations.numpy())
print("time in mins: ", ( end_time - start_time ) / 60 )